# Makefile

## Intuition

Ranging from `pip` installing to doing styling checks using `black .`, we have just started and there are already so many different commands to keep track of.

To help with this, we're going to use a [`Makefile`](https://opensource.com/article/18/8/what-how-makefile) which is a automation tool that organizes our commands. This makes it very easy for us to organize relevant commands as well as organize it for others who may be new to our application.

If you are confused, just imagine `Makefile` as a file to keep track of the commands you want the users to run, organized in a manner that is easily called. Hop on in this tutorial to gain a hands on experience.

https://github.com/drivendata/cookiecutter-data-science/blob/master/%7B%7B%20cookiecutter.repo_name%20%7D%7D/Makefile

```bash
touch Makefile
```

- `line 2`: `SHELL := /bin/bash`
    - `SHELL` is the Makefile variable that sets the preferred shell to use. The default on is `/bin/sh`, so if you need Bash features, you might have to set this variable.
    - [What is Shell Bash?](https://stackoverflow.com/questions/46955861/makefile-what-is-shell-bash)
    - [Choosing Shell in Makefile](https://www.gnu.org/software/make/manual/html_node/Choosing-the-Shell.html)

    

1. We first install the packages needed.

In [ ]:
!make --version

GNU Make 4.1
Built for x86_64-pc-linux-gnu
Copyright (C) 1988-2014 Free Software Foundation, Inc.
License GPLv3+: GNU GPL version 3 or later <http://gnu.org/licenses/gpl.html>
This is free software: you are free to change and redistribute it.
There is NO WARRANTY, to the extent permitted by law.


In [ ]:
!pip install -q black flake8 isort

     |████████████████████████████████| 1.4 MB 4.3 MB/s 
     |████████████████████████████████| 64 kB 2.6 MB/s 
     |████████████████████████████████| 103 kB 57.8 MB/s 
     |████████████████████████████████| 843 kB 41.8 MB/s 
     |████████████████████████████████| 96 kB 6.1 MB/s 
     |████████████████████████████████| 69 kB 7.6 MB/s 
     |████████████████████████████████| 42 kB 832 kB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
markdown 3.3.6 requires importlib-metadata>=4.4; python_version < "3.10", but you have importlib-metadata 4.2.0 which is incompatible.
flask 1.1.4 requires click<8.0,>=5.1, but you have click 8.1.2 which is incompatible.


2. We define a global variable: `BASE_DIR` which points to one level up, its root folder. We verify that it is `/content` (in google colab).

In [ ]:
from pathlib import Path

# Creating Directories
BASE_DIR = Path("__file__").parent.absolute()
print(BASE_DIR)

/content


3. Write a python file named `test.py` into `BASE_DIR`. The file contains the function we talked about just now.

    We now write this function into the file `test.py`. 

In [ ]:
%%writefile {BASE_DIR}/Makefile
# Makefile
SHELL := /bin/bash
VM_NAME := venv_test

say_hello:
	@echo "Hello World"

.PHONY: help
help:
	@echo "Commands:"
	@echo "venv    : creates development environment."
	@echo "style   : runs style formatting."
	@echo "clean   : cleans all unnecessary files."

Writing /content/Makefile


In [ ]:
!make help

Commands:
venv    : creates development environment.
style   : runs style formatting.
clean   : cleans all unnecessary files.


In [ ]:
%%writefile {BASE_DIR}/test.py
def shhq(shhq_member: str = "hn"):
    if shhq_member in ["hn", "cw", "jun", "lh", "lz", "mj", "sz", "wj", "yj", "zj"]:
        return True
    else:
        return False

Writing /content/test.py


4. As detailed in the earlier section, we set some configurations for the formatter `black` and write these in `pyproject.toml` file. 

    Note that we excluded folders like the virtual environments `venv_ae`. As a reminder, we do not want our formatter and linter to check on **every file** in our code base. Even though this example here is not directly applicable, we should take note during production.

In [ ]:
%%writefile {BASE_DIR}/pyproject.toml
# Black formatting
[tool.black]
line-length = 79
include = '\.pyi?$'
exclude = '''
/(
      \.eggs         # exclude a few common directories in the
    | \.git          # root of the project
    | \.hg
    | \.mypy_cache
    | \.tox
    | _build
    | buck-out
    | build
    | dist
    | venv_ae
  )/
'''

Writing /content/pyproject.toml


5. Before we run the `black` formatter, we call `%pycat` to view the python file and take note that in `line 2`, the line length definitely exceeded $79$.

In [ ]:
%pycat {BASE_DIR}/test.py

6. To use these tools that we've configured, we could run these commands individually such as calling `black .` where `.` signifies that the configuration file for that package is in the current directory.

In [ ]:
!black .

reformatted test.py

All done! ✨ 🍰 ✨
1 file reformatted.


7. We see that the console said the files are formatted. We can call `%pycat` once again to check the code is indeed formatted!

In [ ]:
%pycat {BASE_DIR}/test.py

8. We can repeat the steps for our `flake8` file. We will shorten the example here, but for completeness sake we re-initialize `test.py` and see what our `flake8` has to say.

In [ ]:
%%writefile {BASE_DIR}/test.py
def shhq(shhq_member: str = "hn"):
    if shhq_member in ["hn", "cw", "jun", "lh", "lz", "mj", "sz", "wj", "yj", "zj"]:
        return True
    else:
        return False

Overwriting /content/test.py


In [ ]:
%%writefile {BASE_DIR}/.flake8
[flake8]
exclude = venv
ignore = W503, E226 # E501
max-line-length = 79

# E501: Line too long
# W503: Line break occurred before binary operator
# E226: Missing white space around arithmetic operator

Overwriting /content/.flake8


In [ ]:
!flake8

./test.py:2:80: E501 line too long (84 > 79 characters)
./test.py:5:21: W292 no newline at end of file


In the original example, the author ignores `E501: Line too long` clause in order to avoid conflicts with `black`. However, I included it to show as an example. Even though `flake8` highlights the issue, it will not automatically format the code!

It also did not seem to have the `uncessary else after return` statement (perhaps this is under `pylint` and not `flake8`).

9. We have seen the formatter and linter in action. I'll also provide a basic version if you are working in VSCode.

    ```bash
    cd "to your desired directory"
    code . # opens vscode
    touch test.py # touch is mac command to create a new file, upon creation, add in the code
    touch .flake8 # add in the configurations
    touch pyproject.toml # add in the configurations
    black . # runs black config from pyproject and formats code in-place
    isort . # runs isort config from pyproject and formats code in-place
    flake8 # runs flake8
    ```

## Next Steps

Let us see what we can further do to automate this step.